In [51]:
import sqlparse
import textwrap
from matplotlib import pyplot as plt
import sqlparse
import textwrap
from sqlparse.sql import Parenthesis,Function,Identifier, IdentifierList
from sqlparse.tokens import Keyword, Name


In [2]:
def get_fm_sql(file_path:str) -> str:

    '''
    Get formatted sql from file

    Parameters
    ----------
    file_path : str

    Returns
    -------
    indent_str : str
    '''

    with open(file_path, encoding='utf-8') as file:
        content = file.read()
        str_sql = sqlparse.format(content, reindent=True, keyword_case='upper')
        str_sql = str_sql.strip(' \t\n;')
        indent_str = textwrap.indent(str_sql, "  ")
    return indent_str

In [99]:
sql = get_fm_sql('./exam.sql')
print(sql)
parse = sqlparse.parse(sql)

  CREATE TABLE IF NOT EXISTS POS.COMBINED
    (SELECT CASE
                WHEN tStock_In.shop IS NULL THEN tSale_qty.shop
                ELSE tStock_In.shop
            END AS shop,
            CASE
                WHEN tStock_In.prod IS NULL THEN tSale_qty.prod
                ELSE tStock_In.prod
            END AS prod,
            pos110.pname,
            pos110.pluno,
            pos116.lprc,
            CASE
                WHEN tStock_In.stock_qty IS NULL THEN 0
                ELSE tStock_In.stock_qty
            END AS pre_stock_qty,
            CASE
                WHEN tStock_In.in_qty IS NULL THEN 0
                ELSE tStock_In.in_qty
            END AS in_qty,
            CASE
                WHEN tSale_qty.sale_qty IS NULL THEN 0
                ELSE tSale_qty.sale_qty
            END AS sale_qty,
            CASE
                WHEN tSale_qty.sale_qty IS NULL THEN (tStock_In.stock_qty + tStock_In.in_qty)
                WHEN tStock_In.stock_qty IS NULL
             

In [100]:
sql

"  CREATE TABLE IF NOT EXISTS POS.COMBINED\n    (SELECT CASE\n                WHEN tStock_In.shop IS NULL THEN tSale_qty.shop\n                ELSE tStock_In.shop\n            END AS shop,\n            CASE\n                WHEN tStock_In.prod IS NULL THEN tSale_qty.prod\n                ELSE tStock_In.prod\n            END AS prod,\n            pos110.pname,\n            pos110.pluno,\n            pos116.lprc,\n            CASE\n                WHEN tStock_In.stock_qty IS NULL THEN 0\n                ELSE tStock_In.stock_qty\n            END AS pre_stock_qty,\n            CASE\n                WHEN tStock_In.in_qty IS NULL THEN 0\n                ELSE tStock_In.in_qty\n            END AS in_qty,\n            CASE\n                WHEN tSale_qty.sale_qty IS NULL THEN 0\n                ELSE tSale_qty.sale_qty\n            END AS sale_qty,\n            CASE\n                WHEN tSale_qty.sale_qty IS NULL THEN (tStock_In.stock_qty + tStock_In.in_qty)\n                WHEN tStock_In.stoc

##### Test

In [102]:
idfs_list = []
for token in list(parse[0].tokens):
    if token._get_repr_name() == 'Identifier':
        idfs_list.append(token.value)

print(idfs_list)

['POS.COMBINED']


IndexError: list index out of range

In [103]:
FUNCTION_OPERATIONS={'SELECT','DROP','INSERT','UPDATE','CREATE'}
PRECEDES_TABLE_NAME = {'FROM', 'JOIN', 'DESC', 'DESCRIBE', 'WITH'}
RESULT_OPERATIONS = {'UNION', 'INTERSECT', 'EXCEPT', 'SELECT'}
table_names = []


In [104]:
def is_identifier(token):
    return isinstance(token, (IdentifierList, Identifier))

In [105]:
# 檢核DDL字元
def precedes_function_name(token_value):
    for keyword in FUNCTION_OPERATIONS:
        if keyword in token_value:
            return True
    return False

In [106]:
def precedes_table_name(token_value):
    for keyword in PRECEDES_TABLE_NAME:
        if keyword in token_value:
            return True
    return False

In [107]:
def is_result_operation(keyword):
    for operation in RESULT_OPERATIONS:
        if operation in keyword.upper():
            return True
    return False

In [108]:
def process_identifier(identifier):
    # 如果為子查詢，則遞迴處理
    if '(' not in '{}'.format(identifier):
        get_Identifier_keywords_tables(identifier)
        return 
    _extract_table_from_token(identifier)

In [109]:
def get_main_functionsql(statment):
    return statment.get_type()

In [110]:
# 第一层Identifier
def _get_one_Identifier(statment):
    idfr_list = []
    tokens_list = statment.tokens
    for each_token in tokens_list:
        if each_token._get_repr_name() == 'Identifier':
            idfr_list.append(each_token)
    return idfr_list

In [111]:
# 满足库.表形式的identifiers提取即判定为表
def get_Identifier_keywords_tables(identifiers):
    global table_names

    # <tablename abbreviation>
    if len(identifiers.tokens) == 3 and identifiers.tokens[1].value == ' ':
        table_name = identifiers.tokens[0].value
        return table_names.append(table_name)
    
    # <schema.tablename>
    if len(identifiers.tokens) > 1 and identifiers.tokens[1].value == '.':
        schema_name = identifiers.tokens[0].value
        table_name = identifiers.tokens[2].value
        db_table = (schema_name, table_name)
        full_tree = '{}.{}'.format(schema_name, table_name)

        if len(identifiers.tokens) == 3:
            return table_names.append(full_tree)
        
        # <schema.tablename abbreviation>
        else:
            check_space = identifiers.tokens[3].value
            abbr = identifiers.tokens[4].value
            if check_space == ' ':
                return table_names.append(full_tree)
            # <multi schema.tablename>
            full_tree = '{}.{}.{}'.format(schema_name, table_name, abbr)
            return table_names.append(full_tree)
    # <tablename>
    if len(identifiers.tokens) == 1:
        table_name = identifiers.tokens[0].value
        return table_names.append(table_name)

In [112]:
# 提底表表名
def _extract_table_from_token(statment):

    # 確認是否有tokens屬性
    if not hasattr(statment, 'tokens'):
        return
    
    # 可添加多个preceding_token
    table_name_preceding_token = False

    
    for item in statment.tokens:
        # 如果token本身還有子查詢的話就進入遞迴在進行一次剝洋蔥
        if item.is_group and not is_identifier(item): # is_group: 是否有子查詢 # is_identifier: 是否為identifiers
            _extract_table_from_token(item)

        # 檢測token是否為關鍵字(DDL)
        if item.ttype in Keyword:
            # 有关键字的情况下可以判定存在表，那么直接跳到符合的情况下，剩余的token不再判断
            if precedes_table_name(item.value.upper()): # 透過 precedes_table_name 確認是否為table
                table_name_preceding_token = True
                continue
                # 那么直接跳到符合的情况下，剩余的token不再判断
            
        # 若token非關鍵字，跳出迴圈
        if not table_name_preceding_token:
            continue

        # FROM 裏面的子查詢確認
        if item.ttype in Keyword or item.value == ',':
            if (is_result_operation(item.value) or
                    item.value.upper() == 'ON'): # 透過內插DDL文字判斷是否為子查詢 並將 on 視為子查詢
                table_name_preceding_token = False
                continue
            # FROM clause is over
            break

        # 只有identifiers和IdentifierList会有库.表
        if isinstance(item, Identifier):
            process_identifier(item)

        if isinstance(item, IdentifierList):
            for token in item.tokens:
                if is_identifier(token):
                    process_identifier(token)

                # 该方法解析IdentifierList

In [113]:
def blood_table(statment):
    if precedes_function_name(get_main_functionsql(statment)):
        idfr_list = _get_one_Identifier(statment)
        get_Identifier_keywords_tables(idfr_list[0])
    type_name = get_main_functionsql(statment)
    _extract_table_from_token(statment)
    inherit_table = table_names[0]
    root_table = set(table_names[1:])
    if get_main_functionsql(statment) != 'SELECT':
        table_Bloodcurse = '{}->{}'.format(inherit_table, root_table)
        return table_Bloodcurse
    else:
        table_Bloodcurse = set(table_names)
        return table_Bloodcurse

In [114]:
for stmt in parse:
    print(blood_table(stmt))

POS.COMBINED->{'product_stock', 'pos116', 'pos324', 'pos204'}
